In [1]:
import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os

In [2]:
skill = 'Data%20Analysis'
location = 'Amsterdam%2C%20Netherlands'

In [3]:
URL = ('https://jobs.booking.com/careers?location=' + location + '&skill=' + skill)
page = requests.get(URL)

In [4]:
print(page.text)

<!DOCTYPE html>
<html lang="en">

  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=6.0">
    <meta name="referrer" content="strict-origin-when-cross-origin">
    <meta name="_csrf" content="ImViMzJiOGJkMDRhNjFjYjJmNDBiYzJjMTFhMzkzMGE1MGVmZjFmZWEi.FN_jwg.5m1TARl8Hc1pT9RSBtT2vseDPDE">

<link rel="icon" href="https://static.vscdn.net/images/careers/demo/booking/booking_fav_icon_updated.png" sizes="32x32">

  <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1" />
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=6, minimum-scale=1" />
  <meta name="description" content="At Booking.com, we make it easier for everyone to experience the world. We began by taking hotel bookings online over 20 years ago and we&#39;ve been shaping the travel industry ever since"/>
  <meta name="og:image" property="og:image" content="https://static.vscdn.net/images/car

In [5]:
soup = BeautifulSoup(page.content, "html.parser")

In [6]:
results = soup.find(class="position-card-1")

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=6.0" name="viewport"/>
  <meta content="strict-origin-when-cross-origin" name="referrer"/>
  <meta content="ImViMzJiOGJkMDRhNjFjYjJmNDBiYzJjMTFhMzkzMGE1MGVmZjFmZWEi.FN_jwg.5m1TARl8Hc1pT9RSBtT2vseDPDE" name="_csrf"/>
  <link href="https://static.vscdn.net/images/careers/demo/booking/booking_fav_icon_updated.png" rel="icon" sizes="32x32"/>
  <meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible">
   <meta charset="utf-8">
    <meta content="width=device-width, initial-scale=1, maximum-scale=6, minimum-scale=1" name="viewport">
     <meta content="At Booking.com, we make it easier for everyone to experience the world. We began by taking hotel bookings online over 20 years ago and we've been shaping the travel industry ever since" name="description">
      <meta content="https://static.vscdn.net/images/careers/demo/booking/OG.jpg" name="og:image" 

In [ ]:
## ======================= GENERIC FUNCTIONS ======================= ##

def save_jobs_to_csv(jobs_list, filename):
    jobs = pd.DataFrame(jobs_list)
    jobs.to_csv(filename)



## ================== FUNCTIONS FOR INDEED.CO.UK =================== ##

def load_indeed_jobs_div(job_title, location):
    getVars = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date'}
    url = ('https://www.indeed.co.uk/jobs?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    job_soup = soup.find(id="resultsCol")
    return job_soup

def extract_job_information_indeed(job_soup, skills):
    job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
     
    cols = []
    extracted_info = []
    
    
    if 'titles' in skills:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_indeed(job_elem))
        extracted_info.append(titles)                    
    
    if 'companies' in skills:
        companies = []
        cols.append('companies')
        for job_elem in job_elems:
            companies.append(extract_company_indeed(job_elem))
        extracted_info.append(companies)
    
    if 'links' in skills:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_indeed(job_elem))
        extracted_info.append(links)
    
    if 'date_listed' in skills:
        dates = []
        cols.append('date_listed')
        for job_elem in job_elems:
            dates.append(extract_date_indeed(job_elem))
        extracted_info.append(dates)
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    num_listings = len(extracted_info[0])
    
    return jobs_list, num_listings


def extract_job_title_indeed(job_elem):
    title_elem = job_elem.find('h2', class_='title')
    title = title_elem.text.strip()
    return title

def extract_company_indeed(job_elem):
    company_elem = job_elem.find('span', class_='company')
    company = company_elem.text.strip()
    return company

def extract_link_indeed(job_elem):
    link = job_elem.find('a')['href']
    link = 'www.Indeed.co.uk/' + link
    return link

def extract_date_indeed(job_elem):
    date_elem = job_elem.find('span', class_='date')
    date = date_elem.text.strip()
    return date



## ================== FUNCTIONS FOR CWJOBS.CO.UK =================== ##
    

def initiate_driver(location_of_driver, browser):
    if browser == 'chrome':
        driver = webdriver.Chrome(executable_path=(location_of_driver + "/chromedriver"))
    elif browser == 'firefox':
        driver = webdriver.Firefox(executable_path=(location_of_driver + "/firefoxdriver"))
    elif browser == 'safari':
        driver = webdriver.Safari(executable_path=(location_of_driver + "/safaridriver"))
    elif browser == 'edge':
        driver = webdriver.Edge(executable_path=(location_of_driver + "/edgedriver"))
    return driver

def make_job_search(job_title, location, driver):
    driver.get('https://www.cwjobs.co.uk/')
    
    # Select the job box
    job_title_box = driver.find_element_by_name('Keywords')

    # Send job information
    job_title_box.send_keys(job_title)

    # Selection location box
    location_box = driver.find_element_by_id('location')
    
    # Send location information
    location_box.send_keys(location)
    
    # Find Search button
    search_button = driver.find_element_by_id('search-button')
    search_button.click()

    driver.implicitly_wait(5)

    page_source = driver.page_source
    
    job_soup = BeautifulSoup(page_source, "html.parser")
    
    return job_soup


def extract_job_information_cwjobs(job_soup, skills):
    
    job_elems = job_soup.find_all('div', class_="job")
     
    cols = []
    extracted_info = []
    
    if 'titles' in skills:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_cwjobs(job_elem))
        extracted_info.append(titles) 
                           
    
    if 'companies' in skills:
        companies = []
        cols.append('companies')
        for job_elem in job_elems:
            companies.append(extract_company_cwjobs(job_elem))
        extracted_info.append(companies)
    
    if 'links' in skills:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_cwjobs(job_elem))
        extracted_info.append(links)
                
    if 'date_listed' in skills:
        dates = []
        cols.append('date_listed')
        for job_elem in job_elems:
            dates.append(extract_date_cwjobs(job_elem))
        extracted_info.append(dates)    
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    num_listings = len(extracted_info[0])
    
    return jobs_list, num_listings


def extract_job_title_cwjobs(job_elem):
    title_elem = job_elem.find('h2')
    title = title_elem.text.strip()
    return title
 
def extract_company_cwjobs(job_elem):
    company_elem = job_elem.find('h3')
    company = company_elem.text.strip()
    return company

def extract_link_cwjobs(job_elem):
    link = job_elem.find('a')['href']
    return link

def extract_date_cwjobs(job_elem):
    link_elem = job_elem.find('li', class_='date-posted')
    link = link_elem.text.strip()
    return link
